In [1]:
import tensorflow as tf
from tensorflow.python.platform import build_info as tf_build_info
print(tf_build_info.build_info['cudnn_version'])  # Displays cuDNN version


2025-01-05 14:15:59.738695: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-05 14:15:59.901065: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736066759.978226    5474 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736066760.003066    5474 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-05 14:16:00.166621: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

9


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import time

# Check if GPU is available
if tf.test.is_gpu_available():
    print("GPU is available!")
    print(f"GPU Name: {tf.config.list_physical_devices('GPU')}")
else:
    print("No GPU detected or TensorFlow is not using the GPU.")
    exit()

# Load CIFAR-10 dataset (small dataset but useful for testing)
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize to [0, 1]
y_train, y_test = to_categorical(y_train), to_categorical(y_test)  # One-hot encode labels

# Define a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model and measure time
start_time = time.time()
history = model.fit(x_train, y_train, epochs=3, batch_size=64, validation_data=(x_test, y_test))
end_time = time.time()

# Print summary
print("\nTraining completed!")
print(f"Total Training Time: {end_time - start_time:.2f} seconds")
print("Model Performance:")
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


GPU is available!
GPU Name: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1736519733.560129   26316 gpu_device.cc:2022] Created device /device:GPU:0 with 9182 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4070 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9
/home/smayan/Desktop/AI-ML-DS/AI-and-ML-Course/.conda/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/3


I0000 00:00:1736519736.861031   26456 service.cc:148] XLA service 0x74b544004350 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1736519736.861141   26456 service.cc:156]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 SUPER, Compute Capability 8.9
2025-01-10 20:05:36.901467: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1736519737.178937   26456 cuda_dnn.cc:529] Loaded cuDNN version 90600


 37/782 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.1164 - loss: 2.2975

I0000 00:00:1736519739.672074   26456 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


782/782 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.3306 - loss: 1.8109 - val_accuracy: 0.5165 - val_loss: 1.3667
Epoch 2/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step - accuracy: 0.5508 - loss: 1.2645 - val_accuracy: 0.6027 - val_loss: 1.1143
Epoch 3/3
782/782 ━━━━━━━━━━━━━━━━━━━━ 4s 5ms/step - accuracy: 0.6176 - loss: 1.0887 - val_accuracy: 0.6375 - val_loss: 1.0420

Training completed!
Total Training Time: 19.12 seconds
Model Performance:
313/313 - 2s - 5ms/step - accuracy: 0.6375 - loss: 1.0420
Test Loss: 1.042005181312561
Test Accuracy: 0.637499988079071
